In [14]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.models import Model

In [17]:
inputs = Input(shape=(28, 28), name="input_layer")
x = Flatten(name="flatten_layer")(inputs)   
h1 = Dense(32, activation='relu', name="hidden_layer_1")(x)
h2 = Dense(64, activation='relu', name="hidden_layer_2")(h1)
h3 = Dense(32, activation='relu', name="hidden_layer_3")(h2)
outputs = Dense(10, activation='softmax', name="output_layer")(h3)
model = Model(inputs=inputs, outputs=outputs, name="mnist_model")
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [19]:
from tensorflow.keras.datasets import mnist

# Load data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize
x_train = x_train / 255.0
x_test = x_test / 255.0
# Train
model.fit(x_train, y_train, epochs=100, validation_data=(x_test, y_test))
model.save('my_model.keras')

Epoch 1/100
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.9398 - loss: 0.2050 - val_accuracy: 0.9455 - val_loss: 0.1792
Epoch 2/100
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.9575 - loss: 0.1414 - val_accuracy: 0.9609 - val_loss: 0.1267
Epoch 3/100
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.9659 - loss: 0.1123 - val_accuracy: 0.9652 - val_loss: 0.1164
Epoch 4/100
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9708 - loss: 0.0951 - val_accuracy: 0.9655 - val_loss: 0.1137
Epoch 5/100
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9744 - loss: 0.0829 - val_accuracy: 0.9664 - val_loss: 0.1135
Epoch 6/100
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9765 - loss: 0.0744 - val_accuracy: 0.9640 - val_loss: 0.1237
Epoch 7/100
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9790 - loss: 0.0666 - val_accuracy: 0.9699 - val_loss: 0.1097
Epoch 8/100
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.9808 - loss: 0

In [22]:
# Model Accuracy
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test accuracy: {accuracy*100:.2f}%")

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9695 - loss: 0.3026
Test accuracy: 96.95%


In [20]:
import tkinter as tk
from PIL import Image, ImageDraw
import numpy as np
import tensorflow as tf

# ------------------------
# Load your trained model
# ------------------------
model = tf.keras.models.load_model("my_model.keras")

# ------------------------
# GUI Settings
# ------------------------
WIDTH, HEIGHT = 280, 280   # Drawing canvas size
GRID_SIZE = 28             # Model input size
BG_COLOR = "white"         # Canvas background
DRAW_COLOR = "black"       # Drawing color
BRUSH_SIZE = 10            # Brush radius

# ------------------------
# Create Tkinter Window
# ------------------------
root = tk.Tk()
root.title("Digit Recognizer (28x28 FCNN)")

# Create Canvas
canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg=BG_COLOR, cursor="cross")
canvas.grid(row=0, column=0, pady=2, sticky=tk.W, columnspan=2)

# Create a Pillow image to mirror what’s drawn on canvas
image = Image.new("L", (WIDTH, HEIGHT), color=255)  # white background
draw = ImageDraw.Draw(image)

# ------------------------
# Drawing Function
# ------------------------
def paint(event):
    x1, y1 = (event.x - BRUSH_SIZE), (event.y - BRUSH_SIZE)
    x2, y2 = (event.x + BRUSH_SIZE), (event.y + BRUSH_SIZE)
    canvas.create_oval(x1, y1, x2, y2, fill=DRAW_COLOR, outline=DRAW_COLOR)
    draw.ellipse([x1, y1, x2, y2], fill=0)  # black ink in Pillow image

canvas.bind("<B1-Motion>", paint)

# ------------------------
# Prediction Function
# ------------------------
def predict_digit():
    # Resize to 28x28
    img_resized = image.resize((GRID_SIZE, GRID_SIZE))
    img_array = np.array(img_resized)
    img_array = 255 - img_array        # invert colors: white bg → 0, black ink → 255
    img_array = img_array / 255.0      # normalize
    img_array = img_array.reshape(1, 28, 28)  # FCNN input shape

    # Predict
    prediction = model.predict(img_array)
    digit = np.argmax(prediction)
    confidence = np.max(prediction) * 100

    result_label.config(text=f"Prediction: {digit} ({confidence:.2f}%)")

# ------------------------
# Clear Function
# ------------------------
def clear_canvas():
    canvas.delete("all")
    draw.rectangle([0, 0, WIDTH, HEIGHT], fill=255)
    result_label.config(text="Prediction: None")

# ------------------------
# Buttons & Label
# ------------------------
predict_button = tk.Button(root, text="Predict", command=predict_digit)
predict_button.grid(row=1, column=0, pady=2)

clear_button = tk.Button(root, text="Clear", command=clear_canvas)
clear_button.grid(row=1, column=1, pady=2)

result_label = tk.Label(root, text="Prediction: None", font=("Helvetica", 16))
result_label.grid(row=2, column=0, columnspan=2)

root.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
